In [ ]:
#Kalman filter function
function Kalman_filter(i,t,u1,v1)


R=6371000.0;#Earth radius in meters
nt=length(t[i].Times)


#Create the time series of the model 
t_model=t[i].Times[1]:Second(1):t[i].Times[nt]
mt=length(t_model)

dt=1.0;
count1=1;

Kalman_sec=zeros(length(t[i].Times)-1)

x_k=zeros(6,mt)

C=zeros(2,mt)

#Initial conditions from AIS data
x_k[1,1]=t[i].Lon[1]
x_k[2,1]=t[i].Lat[1]
t_current=(t[i].Times[1]-t0).value/1000

current_x=u1(x_k[1,1],x_k[2,1],0.0,t_current)
current_y=v1(x_k[1,1],x_k[2,1],0.0,t_current)
C[1,1]=current_x
C[2,1]=current_y

#current in direction of heading
current_along=sind(t[i].HDG[1])*(current_x)+cosd(t[i].HDG[1])*(current_y)                #Calculate the the current along the heading of the ship
u_ship=(0.514444*(t[i].SOG[1])*(cosd(t[i].COG[1]-t[i].HDG[1])))-current_along   
x_k[3,1]=(u_ship*sind(t[i].HDG[1]))   # u ship clws
x_k[4,1]=(u_ship*cosd(t[i].HDG[1]))   # v ship clws
x_k[5,1]=0.0;
x_k[6,1]=0.0;


#Total velocity vectors - saving for output
stat=zeros(5,mt)
fill!(stat,NaN)
utot=zeros(mt)
vtot=zeros(mt)
inn_squared=zeros(5,mt)
fill!(inn_squared,NaN)
model_stat=zeros(5,mt)
fill!(model_stat,NaN)

ttrack1=zeros(mt)
ttrack1[1]=t_current
utot[1]=current_x+x_k[3,1]+x_k[5,1]
vtot[1]=current_y+x_k[4,1]+x_k[6,1]
coslat=zeros(mt)
fill!(coslat,NaN)
#Identity matrix
Id =1.0* Matrix(I,6, 6)
a=1-(dt/(3600*3));
sigma_x=sqrt(((1-a^2)*0.3^2)/dt)
sigma_y=sqrt(((1-a^2)*0.3^2)/dt)
beta=1;
#Initial Error Covariance matrix
P=Diagonal([0.00021^2,0.00013^2,0.0001^2,0.0001^2,0^2,0^2])

#Measurement uncertainty matrix R
# Lon Lat SOG COG HDG
Rs= Diagonal([0.00021^2,0.00013^2,(1.7e-2*(180/pi))^2,0.051^2,(1.7e-2*(180/pi))^2])




#Calm water speed of the ship
for ti=1:mt-1
    
   #Linearized model x(k+1)=A*x(k)+B*c(k)
   A=[[1 0 (180/(pi*R*cosd(x_k[2,ti])))*dt 0 (180/(pi*R*cosd(x_k[2,ti])))*dt 0]; [0 1 0 (180/(pi*R))*dt 0 (180/(pi*R))*dt];
    [ 0 0 1 0 0 0]; [0 0 0 1 0 0];[0 0 0 0 a 0 ];[0 0 0 0 0 a] ]
   B=[[(180/(pi*R*cosd(x_k[2,ti])))*dt 0];[0  (180/(pi*R))*dt];[0 0]; [0 0];[0 0];[0 0]]
   
    Z=Diagonal([0,0,sqrt(dt),sqrt(dt),sqrt(dt),sqrt(dt)])
       
   
    
   #Forecast    
   x_k[:,ti+1]=A*x_k[:,ti]+B*C[:,ti]
    
        
    #Noise covariance matrix
    beta1=beta^2*(x_k[3,ti+1]-x_k[3,ti])^2
    beta2=beta^2*(x_k[4,ti+1]-x_k[4,ti])^2
    Qs=Diagonal([0,0,(0.0001^2),(0.0001^2),((sigma_x*5)^2+beta1),((sigma_y*5)^2+beta2)])
    
    P=(A*P*A')+(Z*Qs*Z')
   
   
    
  
   
   # Measurement update at ti+1
   time=(t_model[ti+1]-t0).value/1000 #convert to seconds relative to t0
   ttrack1[ti+1]=time    
   C[1,ti+1]=u1(x_k[1,ti+1],x_k[2,ti+1],0.0,ttrack1[ti+1])
   C[2,ti+1]=v1(x_k[1,ti+1],x_k[2,ti+1],0.0,ttrack1[ti+1])    
   utot[ti+1]=C[1,ti+1]+x_k[3,ti+1]+x_k[5,ti+1]                 # u total velocity (calm water+current+ducurrent)
   vtot[ti+1]=C[2,ti+1]+x_k[4,ti+1]+x_k[6,ti+1]               # v total velocity (calm water+current+dvcurrent)
    
    
     
    
   # H Matrix   
   H=[[1 0 0 0 0 0 ];[0 1 0 0 0 0]; 
      [0 0 x_k[4,ti+1]/(x_k[3,ti+1]^2+x_k[4,ti+1]^2)  -x_k[3,ti+1]/(x_k[3,ti+1]^2+x_k[4,ti+1]^2) 0 0];
    [0 0 utot[ti+1]/(sqrt(utot[ti+1]^2+vtot[ti+1]^2)) vtot[ti+1]/(sqrt(utot[ti+1]^2+vtot[ti+1]^2)) utot[ti+1]/(sqrt(utot[ti+1]^2+vtot[ti+1]^2)) vtot[ti+1]/(sqrt(utot[ti+1]^2+vtot[ti+1]^2))]; 
      [0 0 vtot[ti+1]/(utot[ti+1]^2+vtot[ti+1]^2) -utot[ti+1]/(utot[ti+1]^2+vtot[ti+1]^2)  vtot[ti+1]/(utot[ti+1]^2+vtot[ti+1]^2) -utot[ti+1]/(utot[ti+1]^2+vtot[ti+1]^2)]]
     
   #Kalman Gain
   KG=(P*H')*inv((H*P*H'+Rs))
       
   statistics=H*P*H'+Rs
   model_statistics=H*P*H'
    P=0.5*(P+P')
    
   
   F=[[x_k[1,ti+1]]; 
      [x_k[2,ti+1]];
      [atand(x_k[3,ti+1],x_k[4,ti+1])];
      [sqrt(utot[ti+1]^2+vtot[ti+1]^2)];
      [atand(utot[ti+1],vtot[ti+1])]]
    
   # Analysis-correction
    count2=1
    #Check if the model time matches the observations time
    for count=1:nt 
    if (t_model[ti+1]==t[i].Times[count])& (count2==1)
     Y=[[t[i].Lon[count]];
      [t[i].Lat[count]];
      [t[i].HDG[count]];
    [0.51444.*t[i].SOG[count]];
      [t[i].COG[count]]]
            
    
       #Create a Date time Series for Kalman (t_model=t_AIS)     
      Kalman_sec[count1]=(t_model[ti+1]-t0).value/1000
       #Save the cos of Latitude
      coslat[count1]=cosd(t[i].Lat[count]) 
        
      #Save the innovations and the statistics           
      innov_squared=(Y-F).^2
            for s=1:5
                stat[s,ti+1]=statistics[s,s]
                inn_squared[s,ti+1]=innov_squared[s]
                model_stat[s,ti+1]=model_statistics[s,s]
            end
        
            
     #Convert from degrees to meters  
            
     inn_squared[1,ti+1]=((pi.*R.*coslat[count1])./180).^2*inn_squared[1,ti+1]
     inn_squared[2,ti+1]=((pi.*R)./180).^2*inn_squared[2,ti+1]
     stat[1,ti+1]=((pi.*R.*coslat[count1])./180)^2*stat[1,ti+1]
     stat[2,ti+1]=((pi.*R)./180)^2*stat[2,ti+1]
     model_stat[1,ti+1]=((pi.*R.*coslat[count1])./180)^2*model_stat[1,ti+1]
     model_stat[2,ti+1]=((pi.*R)./180)^2*model_stat[2,ti+1]  
           
      
    #Update the measurement        
   x_k[:,ti+1]=x_k[:,ti+1]+KG*(Y-F) #overwrite x_k
             
   P=(Id-KG*H)*P
            
   
 
     count1=count1+1
     count2=0
            
    end 
    end
    
end

return (x_k,model_stat,stat,inn_squared,utot,vtot,t_model,Kalman_sec,C)
    
end

In [ ]:
using Pkg

Pkg.activate("..")

using Particles
using NetCDF
using DataFrames
using CSV
using Plots
using Dates
using Distributions
using LinearAlgebra

In [ ]:
t=[]
#I use only the new Eastbound data here/Change absolute path
push!(t,CSV.read("C:\\Users\\panagiot\\.julia\\Particles\\case_ais_channel\\English Channel Set Oct 2019 Eastbound.txt",header=1));

In [ ]:
#Change format of data headers
for s=1:1
rename!(t[s], Symbol("Latitude (°)")=>Symbol("Lat")) ;
rename!(t[s], Symbol("Longitude (°)")=>Symbol("Lon"));
rename!(t[s], Symbol("SOG (kts)")=>Symbol("SOG"));
rename!(t[s], Symbol("COG (°)")=>Symbol("COG"));
rename!(t[s], Symbol("HDG (°)")=>Symbol("HDG"));
rename!(t[s], Symbol("ROT (°/min)")=>Symbol("ROT"));
#Choose random ship from data
    t[1]=t[1][t[1][:MMSI] .==t[1].MMSI[18], : ];
end

In [ ]:

#Plot heading and then choose manually period with steady heading
plot(t[1].HDG)

In [ ]:

#Choose manually period with steady heading
t[1]=t[1][1:250,:];

In [ ]:
for ti=1:length(t)

   times=t[ti][1]

   ts=[DateTime(t,"dd-mm-yyyy HH:MM:SS") for t=times]

   t[ti].Times=ts

end

In [ ]:
#Fix Time format 

t[1].Times=t[1].Times.+Year(2000);

In [ ]:
cd("P:\\1230882-emodnet_hrsm\\fromPanos\\gtsm3_newdata")
dflow_map=load_nc_info(".",r"gtsm_fine_0002_map.nc")
t0=get_reftime(dflow_map)
interp=load_dflow_grid(dflow_map,50,false);
u1,v1=initialize_interpolation(dflow_map,interp,t0);

t1=get_times(dflow_map,t0);
t0=get_reftime(dflow_map);

In [ ]:
cd("C:\\Users\\panagiot\\.julia\\Particles")

In [ ]:
i=1
x_k,model_stat,stat,inn_squared,utot,vtot,t_model,Kalman_sec,C=Kalman_filter(i,t,u1,v1);

In [ ]:
width=1000

height=1000

Plots.default(:size,[width,height])

gebco_server=WmsServer("emodnet-bathymetry") #gebco or emodnet-bathymetry or open-streetmap

bbox=[0.1,49.5,1.5,50.6]  #area to plot min(Lon), min(Lat), max(Lon), max(Lat)



img=get_map(gebco_server,bbox,width,height)



plot_image(img,bbox)


#plot!(t[2].Lon,t[2].Lat)
plot!(t[1].Lon,t[1].Lat)
plot!(x_k[1,:],x_k[2,:])

In [ ]:
plot(t_model,atand.(x_k[3,:],x_k[4,:]),label="HDG Kalman")
plot!(t[i].Times,t[i].HDG,label="HDG AIS")
title!("Kalman filter")

In [ ]:
plot(t_model,atand.(utot,vtot),label="COG Kalman")
plot!(t[i].Times,t[i].COG,label="COG AIS")
title!("Kalman filter")

In [ ]:
plot(t_model,sqrt.(utot.^2+vtot.^2),label="SOG Kalman")
plot!(t[i].Times,0.514444.*t[i].SOG,label="SOG AIS")
title!("Kalman filter")

In [ ]:
#Statistics and innovations of Kalman filter


Rmse=zeros(5)
sqrtmean=zeros(5)
model_rmse=zeros(5)


for s=1:5
Rmse[s]=sqrt(sum(filter(!isnan,((inn_squared[s,:]))))/length(filter(!isnan,((inn_squared[s,:])))))
sqrtmean[s]=mean(filter(!isnan,sqrt.((stat[s,:]))))
model_rmse[s]=mean(filter(!isnan,sqrt.((model_stat[s,:]))))
end


#Create the Kalman timeseries
Kalman_sec=filter(!iszero,Kalman_sec);
Kalman_time=t0+Kalman_sec.*Second(1);

In [ ]:
#Plot dcurrent
plot(t_model,x_k[5,:],label="du_c")
plot!(t_model,x_k[6,:],label="dv_c")
ylabel!("[m/s]")

In [ ]:
#Plot the longitude RMSE-Predicted RMSE

plot(Kalman_time,filter(!isnan,sqrt.(inn_squared[1,:])),label="RMSE")
plot!(Kalman_time,filter(!isnan,sqrt.(stat[1,:])),label="predicted RMSE")
plot!(Kalman_time,filter(!isnan,sqrt.(model_stat[1,:])),label="model uncertainty")
ylabel!("[meters]")
title!("Longitude")

In [ ]:
#Plot the latitude RMSE-Predicted RMSE

plot(Kalman_time,filter(!isnan,sqrt.(inn_squared[2,:])),label="RMSE")
plot!(Kalman_time,filter(!isnan,sqrt.(stat[2,:])),label="Predicted RMSE")
plot!(Kalman_time,filter(!isnan,sqrt.(model_stat[2,:])),label="model uncertainty")
ylabel!("[meters]")
title!("Latitude")

In [ ]:
#Plot the SOG RMSE-Predicted RMSE

plot(Kalman_time,filter(!isnan,sqrt.(inn_squared[4,:])),label="RMSE")
plot!(Kalman_time,filter(!isnan,sqrt.(stat[4,:])),label="Predicted RMSE")
plot!(Kalman_time,filter(!isnan,sqrt.(model_stat[4,:])),label=" model uncertainty")
ylabel!("[m/s]")
title!("SOG")

In [ ]:
#Plot the COG RMSE-Predicted RMSE


plot(Kalman_time,filter(!isnan,sqrt.(inn_squared[5,:])),label="RMSE")
plot!(Kalman_time,filter(!isnan,sqrt.(stat[5,:])),label="Predicted RMSE")
plot!(Kalman_time,filter(!isnan,sqrt.(model_stat[5,:])),label="model uncertainty")
ylabel!("[degrees]")
title!("COG")

In [ ]:
#Plot the HDG RMSE-Predicted RMSE

plot(Kalman_time,filter(!isnan,sqrt.(inn_squared[3,:])),label="RMSE")
plot!(Kalman_time,filter(!isnan,sqrt.(stat[3,:])),label="Predicted RMSE")
ylabel!("[degrees]")
title!("HDG")
plot!(Kalman_time,filter(!isnan,sqrt.(model_stat[3,:])),label="model uncertainty")

In [ ]:
#Overview of the innovations statistics

Name=["Lon" ;"Lat";"HDG";"SOG";"COG"]
Units=["meters" ;"meters"; "degrees";"m/s";"degrees"]
Uncertainty=[0.00021^2;0.00013^2;(1.7e-2*(180/pi))^2;0.051^2;(1.7*(180/pi))^2]
table=["Type" "RMSE" "Predicted RMSE" "Model Uncertainty" "Measurement Uncertainty" "Units"; Name Rmse sqrtmean model_rmse Uncertainty Units ]

In [ ]:
#Plot data gaps

data_gaps=zeros(length(Kalman_sec)-1)

for dg=1:length(Kalman_sec)-1
    data_gaps[dg]=Kalman_sec[dg+1]-Kalman_sec[dg]
end
plot(log10.(data_gaps),label="Data gaps")
ylabel!("[log seconds]")


In [ ]:
HDG_k=atand.(x_k[3,:],x_k[4,:])
COG_k=atand.(utot,vtot);
SOG_k=sqrt.(utot.^2+vtot.^2);

In [ ]:
Calong=sind.(HDG_k).*C[1,:]+cosd.(HDG_k).*C[2,:];
Ccross=cosd.(HDG_k).*C[1,:]-sind.(HDG_k).*C[2,:];

Calong_c=sind.(HDG_k).*x_k[5,:]+cosd.(HDG_k).*x_k[6,:];
Ccross_c=cosd.(HDG_k).*x_k[5,:]-sind.(HDG_k).*x_k[6,:];

vb=sind.(COG_k.-HDG_k).*((SOG_k));
ub=cosd.(COG_k.-HDG_k).*((SOG_k));

uboat_k=sind.(HDG_k).*x_k[3,:]+cosd.(HDG_k).*x_k[4,:];
vboat_k=cosd.(HDG_k).*x_k[3,:]-sind.(HDG_k).*x_k[4,:];

In [ ]:
plot(t_model,Ccross,label="current")
plot!(t_model,Ccross_c,label="dcurrent")
plot!(t_model,vboat_k,label="ship")
plot!(t_model,vb,label=" Kalman (ship+dcurrent+current))")
plot!(t[i].Times,sind.(t[i].COG.-t[i].HDG).*((t[i].SOG.*0.51444)),label="No Kalman-AIS",legend=:topright)

#plot!(t[i].Times,Ccross_DSCM,label="current_DSCM")

ylabel!("[m/s]")
title!("cross")